In [142]:
# import numpy as np
# import pandas as pd
# from sklearn.preprocessing import StandardScaler, LabelEncoder
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.model_selection import train_test_split
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Dense, Dropout
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.losses import Huber
# from tensorflow.keras.callbacks import EarlyStopping
# import joblib

# # ✅ Load Data & Drop Class Column
# df = pd.read_csv("RequestAllGood.csv")
# if "class" in df.columns:
#     df = df.drop(columns=["class"])

# # ✅ Remove Noisy Features (Keep Only Important Ones)
# FEATURES = [
#     "method", "path", "body", "url_length", "body_length", "path_entropy",
#     "body_entropy", "header_count", "sql_injection_count", "xss_attack_count",
#     "command_injection_count", "directory_traversal_count", "csrf_count"
# ]
# df = df[FEATURES]

# # ✅ Encode Categorical Features
# le_method = LabelEncoder()
# le_path = LabelEncoder()
# df["method"] = le_method.fit_transform(df["method"])
# df["path"] = le_path.fit_transform(df["path"])

# # ✅ Encode `body` Using TF-IDF
# vectorizer = TfidfVectorizer(max_features=100)
# body_tfidf = vectorizer.fit_transform(df["body"].astype(str)).toarray()
# body_tfidf_df = pd.DataFrame(body_tfidf, columns=[f"body_{i}" for i in range(body_tfidf.shape[1])])
# df = df.drop(columns=["body"]).reset_index(drop=True)
# df = pd.concat([df, body_tfidf_df], axis=1)

# # ✅ Normalize Data Using `StandardScaler()`
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(df)

# # ✅ Convert Data into Sequences (`TIME_STEPS = 1` for Feedforward Autoencoder)
# X_train, X_val = train_test_split(X_scaled, test_size=0.2, shuffle=True, random_state=42)

# # ✅ Build Fully Connected Autoencoder
# input_dim = X_train.shape[1]
# input_layer = Input(shape=(input_dim,))
# encoded = Dense(128, activation="swish")(input_layer)
# encoded = Dropout(0.3)(encoded)
# encoded = Dense(64, activation="swish")(encoded)
# decoded = Dense(128, activation="swish")(encoded)
# decoded = Dense(input_dim, activation="sigmoid")(decoded)

# autoencoder = Model(input_layer, decoded)
# autoencoder.compile(optimizer=Adam(learning_rate=0.00005), loss=Huber(delta=1.0))  # ✅ Using Huber Loss

# # ✅ Train Autoencoder with Early Stopping
# early_stopping = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True)
# autoencoder.fit(X_train, X_train,
#                 validation_data=(X_val, X_val),
#                 epochs=400, batch_size=128,  
#                 callbacks=[early_stopping])

# # ✅ Save Model & Encoders
# autoencoder.save("my_model.keras")
# joblib.dump(scaler, "autoencoder_scaler.pkl")
# joblib.dump(le_method, "method_encoder.pkl")
# joblib.dump(le_path, "path_encoder.pkl")
# joblib.dump(vectorizer, "body_vectorizer.pkl")  # Save TF-IDF vectorizer

# print("✅ Training Complete! Model & encoders saved.")

# # ✅ Find Dynamic Anomaly Threshold
# X_test_flat = X_val
# reconstructed = autoencoder.predict(X_test_flat)
# errors = np.mean(np.abs(X_test_flat - reconstructed), axis=1)

# ANOMALY_THRESHOLD = np.percentile(errors, 95)  
# print("🔥 Best Anomaly Threshold:", ANOMALY_THRESHOLD)
# joblib.dump(ANOMALY_THRESHOLD, "anomaly_threshold.pkl")


Epoch 1/400
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.2249 - val_loss: 0.2135
Epoch 2/400
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.2167 - val_loss: 0.2121
Epoch 3/400
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2221 - val_loss: 0.2103
Epoch 4/400
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2159 - val_loss: 0.2079
Epoch 5/400
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.2140 - val_loss: 0.2044
Epoch 6/400
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2069 - val_loss: 0.1989
Epoch 7/400
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1943 - val_loss: 0.1904
Epoch 8/400
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1977 - val_loss: 0.1771
Epoch 9/400
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1769 - val_loss: 0.1578
Epoch 10/400
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1535 - val_loss: 0.1335
Epoch 11/400
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1357 - val_loss: 0.1094
Epoch 12/400
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.

['anomaly_threshold.pkl']

In [29]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber
from tensorflow.keras.callbacks import EarlyStopping
import joblib

In [31]:
# Load data
df = pd.read_csv("RequestAllGood.csv")
if "class" in df.columns:
    df = df.drop(columns=["class"])

In [33]:
# Define features for anomaly detection
FEATURES = [
    "method", "path","body", "url_length", "body_length", "path_entropy",
    "body_entropy", "header_count", "sql_injection_count", "xss_attack_count",
    "command_injection_count", "directory_traversal_count", "csrf_count"
]

In [35]:
# Select features
df = df[FEATURES]

In [37]:
# Handle categorical features, including "body" with empty values
le_method = LabelEncoder()
le_path = LabelEncoder()
le_body = LabelEncoder()

In [39]:
# Encode "method" and "path"
df["method"] = le_method.fit_transform(df["method"].fillna("GET"))  # Default to "GET" for missing
df["path"] = le_path.fit_transform(df["path"].fillna("/"))  # Default to "/" for missing

In [41]:
# Handle "body" (categorical, with empty strings)
df["body"] = df["body"].fillna("")  # Replace NaN or None with empty string
df["body"] = le_body.fit_transform(df["body"])  # Encode empty strings as a valid category

In [43]:
# Combine features (no TF-IDF needed)
X = df[FEATURES].values  # Use only the selected numerical features

In [45]:
# Normalize data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [47]:
# Train-test split
X_train, X_val = train_test_split(X_scaled, test_size=0.2, shuffle=True, random_state=42)

In [49]:
# Build autoencoder
input_dim = X_train.shape[1]  # 12 features
input_layer = Input(shape=(input_dim,))
encoded = Dense(128, activation="swish")(input_layer)
encoded = Dropout(0.3)(encoded)
encoded = Dense(64, activation="swish")(encoded)
decoded = Dense(128, activation="swish")(encoded)
decoded = Dense(input_dim, activation="sigmoid")(decoded)

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer=Adam(learning_rate=0.00005), loss=Huber(delta=1.0))

In [51]:
# Train
early_stopping = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True)
autoencoder.fit(X_train, X_train, validation_data=(X_val, X_val), epochs=500, batch_size=128, callbacks=[early_stopping])

Epoch 1/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.3336 - val_loss: 0.3223
Epoch 2/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3288 - val_loss: 0.3187
Epoch 3/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3232 - val_loss: 0.3148
Epoch 4/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.3220 - val_loss: 0.3100
Epoch 5/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3139 - val_loss: 0.3043
Epoch 6/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3102 - val_loss: 0.2973
Epoch 7/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3063 - val_loss: 0.2886
Epoch 8/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2913 - val_loss: 0.2782
Epoch 9/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2838 - val_loss: 0.2658
Epoch 10/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2723 - val_loss: 0.2519
Epoch 11/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2585 - val_loss: 0.2367
Epoch 12/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2

In [53]:
# Save model and encoders
autoencoder.save("my_model.keras")
joblib.dump(scaler, "autoencoder_scaler.pkl")
joblib.dump(le_method, "AL_method_encoder.pkl")
joblib.dump(le_path, "AL_path_encoder.pkl")
joblib.dump(le_body, "AL_body_encoder.pkl")  # Save body encoder for

['AL_body_encoder.pkl']

In [55]:
# Find anomaly threshold
reconstructed = autoencoder.predict(X_val)
errors = np.mean(np.abs(X_val - reconstructed), axis=1)
ANOMALY_THRESHOLD = np.percentile(errors, 95)
print("🔥 Best Anomaly Threshold:", ANOMALY_THRESHOLD)
joblib.dump(ANOMALY_THRESHOLD, "anomaly_threshold.pkl")

print("✅ Training Complete! Model & encoders saved.")

40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step  
🔥 Best Anomaly Threshold: 0.5676260789062934
✅ Training Complete! Model & encoders saved.
